# VPN con Tailscale

Como en mi caso tengo un doble NAT en casa me es complicado usar una VPN como Wireguard, por lo que una alternativa es usar Tailscale, que por debajo usa Wireguard, pero arregla el problema del doble NAT

## Instalación

No he conseguido que funcione en un contenedor de docker como enrutador de red, que más adelante veremos qué es, por lo que lo vamos a instalar en la raspberry, para ello ejecutamos en ella

``` bash
curl -fsSL https://pkgs.tailscale.com/stable/ubuntu/focal.noarmor.gpg | sudo tee /usr/share/keyrings/tailscale-archive-keyring.gpg >/dev/null
curl -fsSL https://pkgs.tailscale.com/stable/ubuntu/focal.tailscale-keyring.list | sudo tee /etc/apt/sources.list.d/tailscale.list
sudo apt-get update
sudo apt-get install -y tailscale
```

Inicializamos tailscale mediante

``` bash
sudo tailscale up
```

Nos dará un enlace para logearnos, en mi caso lo he hecho con google

## Raspberry como enrutadora de red

Ya tengo instalado tailscale en la raspberry, además me he instalado la aplicación de movil. Pero Tailscale nos da IPs de su subred, por ejemplo si hacemos 

``` bash
tailscale ip -4
```

Obtenemos la IP que nos crea Tailscale en su subred, en mi caso es `100.86.126.54`, por lo que si desde el movil pongo en un navegador `http://100.86.126.54:8123/` me puedo conectar a home assitant, pero esto hace que me tenga que meter en la aplicación, ver la IP de Tailscale de la raspberry y modificar la dirección... No es ágil

Para solucionar esto Tailscale ofrece que un dispositivo funcione como enrutador de subred

![enrutador de red](https://tailscale.com/kb/1019/subnets/subnets.png)

De manera que en mi caso, la raspberry va a funcionar como enrutador de la subred de mi casa. Y de esta manera, desde cualquier dispositivo conectado a Tailscale, podré acceder a cualquier IP de la subred de mi casa a través de su IP, es decir, en mi casa la raspberry tiene la IP 192.168.178.142, por lo que si en el navegador pongo `http://192.168.178.142:8123/` podré acceder a home assistan, y no tengo que cambiar la IP en función de la que me ha dado Tailscale

Para hacer esto ejecutamos en la raspberry los siguientes comandos para habilitar el enrutamiento en IPv4 e IPv6

``` bash
echo 'net.ipv4.ip_forward = 1' | sudo tee -a /etc/sysctl.d/99-tailscale.conf
echo 'net.ipv6.conf.all.forwarding = 1' | sudo tee -a /etc/sysctl.d/99-tailscale.conf
sudo sysctl -p /etc/sysctl.d/99-tailscale.conf
```

Y ahora creamos la ruta a la subred mediante

``` bash
sudo tailscale up --advertise-routes=<SUBRED>/24
```

Tienes que cambiar `<SUBRED>` por la IP de la subred, por ejemplo en mi caso pongo `sudo tailscale up --advertise-routes=192.168.178.0/24`

Ya hemos configurado la raspberry como enrutadora, ahora tenemos que ir a la [página de las máquinas de Tailscale](https://login.tailscale.com/admin/machines) y darle a los tres puntos de la derecha de la máquina `raspberry` para que se abra el menú de opciones y a continuación pinchamos a `Edit route settings...`, se nos abrirá una ventana y habilitamos el switch `Subnet routes`

Puede que nos salga un mensaje indicando que la auth key temporal se acabará, por lo que volvemos a pinchar en los tres puntos de la máquina `raspberry` para abrir el menú de opciones y pinchamos en `Disable key expiry` si aparece (silo que aparede es `Enable key espirit` **no** pinchamos). De esta manera hemos deshabilitado que se caduque la key de autentificación

Una vez hecho esto, desde cualquier dispositivo de Tailscale podemos acceder a cualquier dispositivo de la subred de nuestra casa

## Configurar la raspberry como un nodo de salida

Si en el momento que nos conectamos a desde un cliente a la VPN de Tailscale, queremos poder navegar por internet, tenemos que configurar la raspberry como un nodo de salida, para ello ejecutamos en la raspberry

``` bash
sudo tailscale set --advertise-exit-node
```

Ya hemos configurado la raspberry nodo de salida, ahora tenemos que ir a la [página de las máquinas de Tailscale](https://login.tailscale.com/admin/machines) y darle a los tres puntos de la derecha de la máquina `raspberry` para que se abra el menú de opciones y a continuación pinchamos a `Edit route settings...`, se nos abrirá una ventana y habilitamos el switch `Exit node`

Una vez hecho esto, desde cualquier dispositivo de Tailscale podemos acceder a internet a través de la raspberry